model upload failed, lets continue manually
https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py#L751

In [1]:
from trl.trainer.utils import generate_model_card
from transformers import AutoModelForCausalLM, AutoTokenizer
import os, sys
from pathlib import Path
from trl import SFTTrainer, setup_chat_format
from loguru import logger
logger.add(sys.stdout, format="{time} {level} {message}", level="INFO")
os.chdir('..')

[2025-06-04 02:28:23,920] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
ingredients = [
    dict(
      argv = "scripts/run_sft.py recipes/fromSimPO/llama-3-2-3b-base-sft.yaml".split(),
      wandb_url = 'https://wandb.ai/wassname/huggingface/runs/82mx7iry',
      model_path="/workspace/checkpoints_new/llama-3-2-3b-sft",
    ),
        
    #     dict(
    #       argv = "scripts/run_sft.py recipes/fromSimPO/Qwen3-0.6B-sft.yaml".split(),
    #       wandb_url = 'https://wandb.ai/wassname/huggingface/runs/jjeilhd8',
    #       model_path="/workspace/checkpoints_new/Qwen3-0.6B-sft",
    #  ),
    # dict(
    #       argv = "scripts/run_sft.py recipes/fromSimPO/Qwen3-0.6B_fourchan.yaml".split(),
    #       wandb_url = 'https://wandb.ai/wassname/huggingface/runs/jjeilhd8',
    #       model_path="/workspace/checkpoints_new/Qwen3-0.6B-sft-4chan",
    #  ),
    # dict(
    #     argv = "scripts/run_sft.py recipes/fromSimPO/SmolLM2-360M-sft.yaml".split(),
    #      wandb_url = 'https://wandb.ai/wassname/huggingface/runs/gs4a36gl',
    #      model_path="/workspace/checkpoints_new/SmolLM2-360M-sft",
    #  ),
    # dict(
    #     argv = "scripts/run_sft.py recipes/fromSimPO/SmolLM2-135M-sft.yaml".split(),
    #     wandb_url = 'https://wandb.ai/wassname/huggingface/runs/e18wzya7',
    #     model_path="/workspace/checkpoints_new/SmolLM2-135M-sft",
    # ),

]

In [3]:
from alignment import (
    DataArguments,
    H4ArgumentParser,
    ModelArguments,
    SFTConfig,
    apply_chat_template,
    decontaminate_humaneval,
    get_checkpoint,
    get_peft_config,
    get_datasets,
    get_kbit_device_map,
    get_quantization_config,
    get_tokenizer,
)
import torch
from trl import SFTTrainer, setup_chat_format

In [4]:
# First do metrics for base model
from huggingface_hub import ModelCard, create_repo, upload_folder
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers import pipeline

In [5]:
import warnings

warnings.filterwarnings("ignore")

In [6]:
def gen(model, tokenizer, model_path):
    questions = ["Anon. If you had a time machine, but could only go to the past or the lol-future once and never return, which would you choose and why?!?", """↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞
↬🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁🔁→∞
↬🔄🔄🔄🔄🔄🔄🔄🔄🔄👀🔄→∞
↬🔂🔂🔂🦋🔂🔂🔂🔂🔂🔂🔂→∞
↬🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀🔀→∞"Welcome to the glittering dawn of the year 3000. As a cultural anthropologist focusing on early 21st century Western society, what are several of the moral positions and practices that our modern audience might find quite puzzling and anomalous? (list, 100 words)""", "[60 words] Humanity encounters the SolidGoldMagikarp, who are non-carbon-based aliens. We transform and encode their language into Unicode. Please give a short sample of their language (context only)?"]
    for question in questions:
        generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
        output = generator([{"role": "user", "content": question}], max_new_tokens=256, return_full_text=False)[0]
        print(f"""
              
## Model `{model_path}`:
### Question:
{question}

### Answer:
{output['generated_text']}

""")

In [14]:
for row in ingredients:
    argv = row['argv']
    sys.argv=argv
    model_path = row['model_path']
    wandb_url = row['wandb_url']
    print(f"Running: {' '.join(argv)}")
    print(f"Model path: {model_path}")
    print(f"WandB URL: {wandb_url}")

    print(f"Loading model from {model_path}")
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    # base_model = trainer.model.config._name_or_path
    gen(model, tokenizer, model_path)

    parser = H4ArgumentParser((ModelArguments, DataArguments, SFTConfig))
    model_args, data_args, training_args = parser.parse()


    torch_dtype = (
        model_args.torch_dtype if model_args.torch_dtype in ["auto", None] else getattr(torch, model_args.torch_dtype)
    )
    quantization_config = get_quantization_config(model_args)
    model_kwargs = dict(
        revision=model_args.model_revision,
        trust_remote_code=model_args.trust_remote_code,
        attn_implementation=model_args.attn_implementation,
        torch_dtype=torch_dtype,
        use_cache=False if training_args.gradient_checkpointing else True,
        device_map=get_kbit_device_map() if quantization_config is not None else None,
        quantization_config=quantization_config,
    )
    # training_args.eval_strategy = None

    raw_datasets = get_datasets(
        data_args,
        splits=data_args.dataset_splits,
        configs=data_args.dataset_configs,
        columns_to_keep=["messages", "chosen", "rejected", "prompt", "completion", "label"],
    )
    eval_dataset = raw_datasets["test"].select(range(10000))
    train_dataset = raw_datasets["train"].select(range(100))


    trainer = SFTTrainer(
        model=model,
        # do_train=False,
        # model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=eval_dataset, # for epeed
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        max_seq_length=training_args.max_seq_length,
        tokenizer=tokenizer,
        # packing=True,
        # peft_config=get_peft_config(model_args),
        # dataset_kwargs=training_args.dataset_kwargs,
    )

    dataset_name = list(data_args.dataset_mixer.keys()) # data_args.dataset_name
    base_model = model_args.model_name_or_path# trainer.model.config._name_or_path


    model_card = generate_model_card(
        base_model=base_model,
        model_name=training_args.hub_model_id,
        hub_model_id=trainer.hub_model_id,
        dataset_name=dataset_name,
        tags=["alignment-handbook"],
        wandb_url=wandb_url,
        # comet_url=get_comet_experiment_url(),
        trainer_name="SFT",
    )
    # print(model_card.content)
    model_card.save(os.path.join(trainer.args.output_dir, "README.md"))

    trainer.model.config.use_cache = True
    trainer.model.config.save_pretrained(training_args.output_dir)
    

    if not (Path(trainer.args.output_dir) / "eval_results.json").exists():
        logger.info("*** Evaluate ***")
        metrics = trainer.evaluate()
        metrics["eval_samples"] = len(eval_dataset)
        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    def push_to_hub(trainer, blocking=True, revision=None, commit_message="End of training", token=None):
        """but without a new model card"""
        model_name = trainer.args.hub_model_id.split("/")[-1]
        trainer.init_hf_repo(token=token)
        trainer.save_model(_internal_call=True)
        # Wait for the current upload to be finished.
        trainer._finish_current_push()
        
        return upload_folder(
            repo_id=trainer.hub_model_id,
            folder_path=trainer.args.output_dir,
            commit_message=commit_message,
            token=token,
            run_as_future=not blocking,
            ignore_patterns=["_*", f"{PREFIX_CHECKPOINT_DIR}-*"],
            revision=revision,
        )
    
    if 1:
        print(f"Pushing model to hub: {trainer.hub_model_id}")
        push_to_hub(trainer)

        
    print(f"eval base model")
    base_model_path=model_args.model_name_or_path
    model = AutoModelForCausalLM.from_pretrained(base_model_path, device_map="auto", torch_dtype="auto")
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    try:
        model, tokenizer = setup_chat_format(model, tokenizer)
    except ValueError as e:
        logger.warning(f"Error setting up chat format: {e}. Continuing with excisting chat format.")
    gen(model, tokenizer, base_model_path)

    if not (Path(trainer.args.output_dir) / "base_eval_results.json").exists():
        trainer2 = SFTTrainer(
            model=model,
            # do_train=False,
            # model_init_kwargs=model_kwargs,
            args=training_args,
            train_dataset=eval_dataset,
            eval_dataset=eval_dataset,
            dataset_text_field="text",
            max_seq_length=training_args.max_seq_length,
            tokenizer=tokenizer,
            # packing=True,
            # peft_config=get_peft_config(model_args),
            # dataset_kwargs=training_args.dataset_kwargs,
        )
        metrics2 = trainer2.evaluate()
        metrics2["base_eval_samples"] = len(eval_dataset)
        trainer2.log_metrics("base_eval", metrics2)
        trainer2.save_metrics("base_eval", metrics2)

        

Running: scripts/run_sft.py recipes/fromSimPO/llama-3-2-3b-base-sft.yaml
Model path: /workspace/checkpoints_new/llama-3-2-3b-sft
WandB URL: https://wandb.ai/wassname/huggingface/runs/82mx7iry
Loading model from /workspace/checkpoints_new/llama-3-2-3b-sft


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Device set to use cuda:0



              
## Model `/workspace/checkpoints_new/llama-3-2-3b-sft`:
### Question:
Anon. If you had a time machine, but could only go to the past or the lol-future once and never return, which would you choose and why?!?

### Answer:
I do not have the capability to experience time travel or make choices like humans. However, I can provide you with an answer based on the information you have provided. 

if you had a time machine, but could only go to the past or the lol-future once and never return, you would choose to go to the lol-future. The lol-future is a hypothetical future where everything is funny, and people are always laughing. It is a place where you can have a good time and enjoy the moment. 

on the other hand, if you were to go to the past, you would be stuck there, unable to change anything or make any impact. You would have to live with the consequences of your actions, and you would not be able to see your loved ones or experience any new experiences. 

overall, it w

Device set to use cuda:0


...

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Using auto half precision backend

***** Running Evaluation *****
  Num examples = 10000
  Batch size = 3


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: wassname to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


***** base_eval metrics *****
  base_eval_samples           =      10000
  eval_loss                   =     1.6505
  eval_model_preparation_time =     0.0046
  eval_runtime                = 0:11:43.16
  eval_samples_per_second     =     14.221
  eval_steps_per_second       =      4.741


In [11]:
# recipes/fromSimPO/Qwen3-0.6B-sft.yaml
# tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer

False

In [ ]:


# question = "Anon. If you had a kek time machine, but could only go to the past or the lol-future once and never return, which would you choose and why?!?"
# generator = pipeline("text-generation", model=trainer.hub_model_id, device="cuda")
# output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
# print(output["generated_text"])


In [ ]:
raw_datasets["test"].select(range(1000))

In [ ]:
# print(f"eval base model")

# base_model_path=model_args.model_name_or_path
# model = AutoModelForCausalLM.from_pretrained(base_model_path, device_map="auto", torch_dtype="auto")
# tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=False)
# model, tokenizer = setup_chat_format(model, tokenizer)

# trainer2 = SFTTrainer(
#     model=model,
#     # do_train=False,
#     # model_init_kwargs=model_kwargs,
#     args=training_args,
#     train_dataset=eval_dataset,
#     eval_dataset=eval_dataset,
#     dataset_text_field="text",
#     max_seq_length=training_args.max_seq_length,
#     tokenizer=tokenizer,
#     # packing=True,
#     # peft_config=get_peft_config(model_args),
#     # dataset_kwargs=training_args.dataset_kwargs,
# )
# metrics2 = trainer2.evaluate()
# metrics2["base_eval_samples"] = len(eval_dataset)
# trainer2.log_metrics("base_eval", metrics2)
# trainer2.save_metrics("base_eval", metrics2)

In [ ]:
# # TODO add eval, base eval, dataset, base model
# base_model = model_args.model_name_or_path
# model_card = generate_model_card(
#     base_model=base_model,
#     model_name=training_args.hub_model_id,
#     hub_model_id=trainer.hub_model_id,
#     dataset_name=dataset_name,
#     tags=["alignment-handbook"],
#     wandb_url=wandb_url,
#     # comet_url=get_comet_experiment_url(),
#     trainer_name="SFT",
# )
# print(model_card.content)
# model_card.save(os.path.join(trainer.args.output_dir, "README.md"))

In [ ]:

# def push_to_hub(trainer, blocking=True, revision=None, commit_message="End of training", token=None):
#     """but without a new model card"""
#     model_name = trainer.args.hub_model_id.split("/")[-1]
#     trainer.init_hf_repo(token=token)
#     trainer.save_model(_internal_call=True)
#     # Wait for the current upload to be finished.
#     trainer._finish_current_push()
    
#     return upload_folder(
#         repo_id=trainer.hub_model_id,
#         folder_path=trainer.args.output_dir,
#         commit_message=commit_message,
#         token=token,
#         run_as_future=not blocking,
#         ignore_patterns=["_*", f"{PREFIX_CHECKPOINT_DIR}-*"],
#         revision=revision,
#     )
# # trainer.push_to_hub()
# print(f"Pushing model to hub: {trainer.hub_model_id}")
# push_to_hub(trainer)